### 손가락 포즈로 스마트 조명 제어 웹앱

In [1]:
from flask import Flask, render_template, Response
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import cv2
import mediapipe as mp
from openai import OpenAI
import os
from dotenv import load_dotenv

# 한글 폰트 경로 
font_path = "C:/Windows/Fonts/malgun.ttf"  # 맑은 고딕 폰트

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

app = Flask(__name__)

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

# 비디오 캡처
cap = cv2.VideoCapture(0)

# 초기 조명 상태
light_status = "OFF"
last_message = "조명이 꺼져 있습니다."

# 손가락 접힌 상태 계산 함수
def count_open_fingers(hand_landmarks):
    fingers = []

    # 엄지
    if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # 검지, 중지, 약지, 새끼
    tips = [mp_hands.HandLandmark.INDEX_FINGER_TIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
            mp_hands.HandLandmark.RING_FINGER_TIP,
            mp_hands.HandLandmark.PINKY_TIP]

    dips = [mp_hands.HandLandmark.INDEX_FINGER_DIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_DIP,
            mp_hands.HandLandmark.RING_FINGER_DIP,
            mp_hands.HandLandmark.PINKY_DIP]

    for tip, dip in zip(tips, dips):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[dip].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers.count(1)

# OpenAI로 문구 생성
def generate_light_message(status):
    prompt = f"조명 상태가 '{status}'입니다. 사용자를 위한 짧은 한 문장의 인삿말을 만들어줘."
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


def draw_text_with_pil(img, text, position, font_size=30, color=(255, 0, 0)):
    # OpenCV 이미지 → PIL 이미지 변환
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=color)

    # PIL 이미지 → OpenCV 이미지 변환
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def gen_frames():
    global light_status, last_message

    while True:
        success, frame = cap.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                fingers_open = count_open_fingers(hand_landmarks)

                previous_status = light_status

                if fingers_open == 0:
                    light_status = "OFF"
                elif fingers_open == 5:
                    light_status = "ON"
                elif fingers_open == 2:
                    light_status = "MOOD"

                if light_status != previous_status:
                    try:
                        last_message = generate_light_message(light_status)
                    except Exception as e:
                        last_message = f"오류 발생: {e}"

        # 한글을 PIL로 그리기 
        frame = draw_text_with_pil(frame, f"조명 상태: {light_status}", (10, 30), font_size=30, color=(255, 0, 0))
        frame = draw_text_with_pil(frame, last_message, (10, 80), font_size=18, color=(0, 255,0))
       
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2025 16:25:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 16:25:58] "GET /video_feed HTTP/1.1" 200 -
